# Stock Comparison Tool

Trying to implement something similar to https://www.barchart.com/myBarchart/quotes/SPY/interactive-chart but in Gradio.

The code in the first code block of this notebook is executed when the `app.py` file is run. This notebook is used for the simplicity of interactive development and testing.

In [1]:
from datetime import timedelta

import gradio as gr
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import yfinance as yf

# retrieve historical stock prices
tickers = ["AAPL", "MSFT", "GOOGL"]
df = yf.download(
    tickers,
    interval="1d",
    period="max",
    progress=False,
).Close.bfill()
df

Ticker,AAPL,GOOGL,MSFT
Date,,,
1980-12-12,0.098834,2.501941,0.059946
1980-12-15,0.093678,2.501941,0.059946
1980-12-16,0.086802,2.501941,0.059946
1980-12-17,0.088951,2.501941,0.059946
1980-12-18,0.091530,2.501941,0.059946
...,...,...,...
2024-12-20,254.490005,191.410004,436.600006
2024-12-23,255.270004,194.630005,435.250000
2024-12-24,258.200012,196.110001,439.329987


### Gradio plot -- this works but is slow to update

In [2]:
def plot_asset_prices(period, shift):
    end_date = df.index[-1] - timedelta(days=shift)
    start_date = end_date - timedelta(days=period)
    df_normalized = df[(df.index >= start_date) & (df.index <= end_date)]
    df_normalized = df_normalized / df_normalized.iloc[0] - 1
    df_normalized.reset_index(names="Date", inplace=True)
    return gr.LinePlot(
        value=pd.melt(df_normalized, id_vars=["Date"], var_name="Asset", value_name="Price"),
        x="Date",
        y="Price",
        color="Asset",
        title="Normalized Asset Prices",
        y_title="Relative Change",
        x_label_angle=45,
        # height=600,
    )


with gr.Blocks() as demo:
    plot = plot_asset_prices(365, 0)
    with gr.Row():
        period = gr.Radio(
            choices=[
                ("5y", 5 * 365),
                ("3y", 2 * 365),
                ("2y", 2 * 365),
                ("1y", 365),
                ("6mo", 182),
                ("1mo", 30),
                ("1w", 7),
            ],
            value=365,
            label="Period",
        )
        shift = gr.Slider(minimum=0, maximum=365, value=0, label="End Date")
    period.change(plot_asset_prices, inputs=[period, shift], outputs=plot)
    shift.change(plot_asset_prices, inputs=[period, shift], outputs=plot)

demo.launch()

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


### Plotly plot -- this is super responsive but does not update

In [3]:
dg = pd.DataFrame(np.random.randn(1000, 3).cumsum(axis=0))

fig = go.Figure()
for col in dg:
    fig.add_trace(go.Scatter(x=dg.index, y=dg[col], mode="lines", name=col))
fig.update_layout(
    xaxis_rangeslider_visible=True,
)